In [1]:
import os
import os.path as osp
import time
import math
import torch
import numpy as np

os.chdir('../')

import mmcv    
from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
from mmengine import fileio
from mmdet3d.utils import replace_ceph_backend
from mmengine.structures import InstanceData
from mmdet.models.layers import inverse_sigmoid

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-14e53p0w because the default path (/home/hello/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# cfg = Config.fromfile('my_projects/CMT/configs/cmt_nus.py')
# cfg = Config.fromfile('my_projects/CMT/configs/cmt_coda.py')
# cfg = Config.fromfile('my_projects/MVXNet/configs/mvxnet_coda.py')
# cfg = Config.fromfile('my_projects/BEVFusion/configs/bevfusion_lidar_cam_coda.py')
cfg = Config.fromfile('my_projects/CenterPoint/configs/centerpoint_coda.py')
cfg.work_dir = osp.abspath('./test/work_dirs')
runner = Runner.from_cfg(cfg)
# runner.load_checkpoint('ckpts/CMT/epoch_29.pth')

12/21 11:50:14 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 110875079
    GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
    CUDA_HOME: /usr/local/cuda-12.5
    NVCC: Cuda compilation tools, release 12.5, V12.5.40
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=comput

In [ ]:
runner.model.eval()
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
        
    metrics = runner.val_evaluator.evaluate(len(runner.val_dataloader.dataset))

In [3]:
for data_batch in runner.test_dataloader:
    break
data_batch = runner.model.data_preprocessor(data_batch, training=False)
batch_inputs_dict = data_batch['inputs']
batch_data_samples = data_batch['data_samples']
imgs = batch_inputs_dict.get('imgs', None)
points = batch_inputs_dict.get('points', None)
img_metas = [item.metainfo for item in batch_data_samples]
gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]

12/21 11:50:21 - mmengine - INFO - ------------------------------
12/21 11:50:21 - mmengine - INFO - The length of test dataset: 563
12/21 11:50:21 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Car        | 1036   |
| Pedestrian | 3933   |
| Cyclist    | 59     |
+------------+--------+


In [ ]:
runner.model.pts_bbox_head.bbox_coder

In [4]:
# 训练过程
runner.model.train()
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

12/21 11:50:27 - mmengine - INFO - ------------------------------
12/21 11:50:27 - mmengine - INFO - The length of training dataset: 563
12/21 11:50:27 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Car        | 1036   |
| Pedestrian | 3933   |
| Cyclist    | 59     |
+------------+--------+


In [5]:
losses

{'task0.loss_heatmap': tensor(483.4712, device='cuda:0', grad_fn=<MulBackward0>),
 'task0.loss_bbox': tensor(1.3783, device='cuda:0', grad_fn=<MulBackward0>),
 'task1.loss_heatmap': tensor(105.5408, device='cuda:0', grad_fn=<MulBackward0>),
 'task1.loss_bbox': tensor(1.2320, device='cuda:0', grad_fn=<MulBackward0>),
 'task2.loss_heatmap': tensor(1452.7211, device='cuda:0', grad_fn=<MulBackward0>),
 'task2.loss_bbox': tensor(1.1575, device='cuda:0', grad_fn=<MulBackward0>)}

In [6]:
# 验证过程
runner.model.eval()
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
        break

12/21 11:50:32 - mmengine - INFO - ------------------------------
12/21 11:50:32 - mmengine - INFO - The length of test dataset: 563
12/21 11:50:32 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Car        | 1036   |
| Pedestrian | 3933   |
| Cyclist    | 59     |
+------------+--------+


/home/hello/anaconda3/envs/python310/lib/python3.10/site-packages/mmdet3d/models/task_modules/coders/centerpoint_bbox_coders.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(


In [ ]:
pred = outputs[0].get('pred_instances_3d')
pred['scores_3d'].shape, pred['labels_3d'].shape, pred['bboxes_3d'].shape

In [ ]:
runner.model.train()
with torch.no_grad():
    data_batch = next(iter(runner.train_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('gt_instances_3d')['bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('gt_instances_3d')['labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

In [ ]:
outs['dn_mask_dict']

In [ ]:
runner.model.eval()
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

In [ ]:
class_names = ['Car', 'Pedestrian', 'Cyclist']
dist_thresh = [0.15, 0.25, 0.5, 1.0]
pred_scores_list = [[]] * len(class_names)
is_tp_list = [[[]] * len(dist_thresh)] * len(class_names)

In [ ]:
# 计算mAP和AR
class_names = ['Car', 'Pedestrian', 'Cyclist']
dist_thresh = [0.15, 0.25, 0.5, 1.0]
pred_scores_list = [[] for _ in range(len(class_names))]
is_tp_list = [[[] for _ in range(len(dist_thresh))] for _ in range(len(class_names))]
num_gt = [0 for _ in range(len(class_names))]
count = 0
runner.model.eval()

with torch.no_grad():
    for data_batch in runner.test_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        for i in range(len(outputs)):
            pred_bboxes = outputs[i].get('pred_instances_3d')['bboxes_3d'].tensor
            pred_labels = outputs[i].get('pred_instances_3d')['labels_3d']
            pred_scores = outputs[i].get('pred_instances_3d')['scores_3d']
            gt_bboxes = data_batch['data_samples'][i].get('eval_ann_info')['gt_bboxes_3d'].tensor
            gt_labels = data_batch['data_samples'][i].get('eval_ann_info')['gt_labels_3d']
            for c in range(len(class_names)):
                gt_bboxes_c = gt_bboxes[gt_labels == c]
                if len(gt_bboxes_c) == 0:
                    continue
                pred_bboxes_c = pred_bboxes[pred_labels == c]
                pred_scores_c = pred_scores[pred_labels == c]
                dist_matrix = torch.cdist(pred_bboxes_c[:, :3], gt_bboxes_c[:, :3]).repeat(len(dist_thresh), 1).reshape(len(dist_thresh), len(pred_bboxes_c), len(gt_bboxes_c))
                is_tp = torch.zeros(len(dist_thresh), len(pred_bboxes_c))
                for j in range(len(dist_thresh)):
                    for k in range(len(pred_bboxes_c)):
                        if dist_matrix[j][k].min() < dist_thresh[j]:
                            is_tp[j][k] = 1
                            dist_matrix[j][:, dist_matrix[j][k].argmin()] = 1000
                for j in range(len(dist_thresh)):
                    is_tp_list[c][j].append(is_tp[j])
                pred_scores_list[c].append(pred_scores_c)
                num_gt[c] += len(gt_bboxes_c)
        count += 1
        if count % 100 == 0:
            print(count)

In [ ]:
all_class_ap = []
all_class_cum_precision = []
all_class_cum_recall = []
for c in range(len(class_names)):
    pred_scores = torch.cat(pred_scores_list[c], dim=0)
    cum_precision_list = []
    cum_recall_list = []
    ap_list = []
    _, sort_idx = pred_scores.sort(descending=True)
    print('sum of preds:', len(pred_scores))

    for i in range(len(dist_thresh)):
        is_tp = torch.cat(is_tp_list[c][i], dim=0)
        is_tp = is_tp[sort_idx]
        cum_tp = torch.cumsum(is_tp, dim=0)
        cum_precision = cum_tp / torch.arange(1, len(pred_scores) + 1)
        cum_recall = cum_tp / num_gt[c]
        
        max_precision = torch.zeros_like(cum_precision)
        max_precision[-1] = cum_precision[-1]
        for j in range(len(pred_scores) - 2, -1, -1):
            max_precision[j] = torch.max(max_precision[j+1], cum_precision[j])
        
        div = []
        for j in range(len(pred_scores)):
            if j == 0 or pred_scores[j] != pred_scores[j-1]:
                div.append(j)

        ap = 0
        for j in div:
            if cum_recall[j] > 0.1:
                if j == 0:
                    ap += max_precision[j] * cum_recall[j]
                else:
                    ap += max_precision[j] * (cum_recall[j] - cum_recall[j-1])

        ap_list.append(ap)
        cum_precision_list.append(cum_precision)
        cum_recall_list.append(cum_recall)
        print('class:', class_names[c], 'dist_thresh:', dist_thresh[i], 'ap:', ap)
        
    all_class_ap.append(ap_list)
    all_class_cum_precision.append(cum_precision_list)
    all_class_cum_recall.append(cum_recall_list)